In [2]:
%matplotlib

import pandas as pd
import os
from backtesting import Updater
# import talib
from binance import Client
from sklearn.preprocessing import MinMaxScaler 
# from settings import key, secret
import backtrader as bt 
client = Client()


Using matplotlib backend: TkAgg


In [3]:
all_data = Updater()
# all_data.update_all_data()
all_data.get_all_data(min_correlation=0.83)

In [4]:
all_data.final_df

,Currency1,Currency2,Correlation,Cointegration,Criticals,Ratio
15,MANAUSDT,SANDUSDT,0.894680,-4.181753,-3.926781,0.851756
10,ARDRUSDT,WAXPUSDT,0.882577,-4.087809,-3.926781,1.150877
0,IOTAUSDT,ENJUSDT,0.861487,-5.263174,-3.926781,0.558943
16,TLMUSDT,SLPUSDT,0.851384,-5.612896,-3.926781,6.904762
6,ENJUSDT,WAXPUSDT,0.851237,-5.943331,-3.926781,5.486550
7,IOTXUSDT,ALICEUSDT,0.845830,-4.356646,-3.926781,0.015256
11,STMXUSDT,WAXPUSDT,0.843693,-7.797118,-3.926781,0.082643
8,IOTXUSDT,TLMUSDT,0.843554,-4.933398,-3.926781,1.233621
1,IOTAUSDT,ZENUSDT,0.838651,-4.557677,-3.926781,0.019153
14,ZENUSDT,ALICEUSDT,0.837720,-4.052619,-3.926781,7.297441


In [8]:
df = all_data.useful_df['MANAUSDT'] - all_data.useful_df['SANDUSDT'] * 0.894680
# df = MinMaxScaler().fit_transform(df.values)
df = df.values.reshape(1,-1)
df

ValueError: cannot reshape array of size 365 into shape (0,1)

In [ ]:
dfs = [full_df[x['Currency1']] - full_df[x['Currency2']] * x['Ratio'] for __, x in td_view.iterrows()]
dfs_fit = list()

for df in dfs:
    df[df.columns] = MinMaxScaler().fit_transform(df[df.columns])
    dfs_fit.append(df)
    
dfs_fit

[                         Open      High       Low     Close
 Time                                                       
 2022-03-22 00:00:00  0.165341  0.116793  0.159980  0.122366
 2022-03-22 04:00:00  0.121847  0.099173  0.094404  0.097849
 2022-03-22 08:00:00  0.097316  0.012213  0.073949  0.019588
 2022-03-22 12:00:00  0.018483  0.050289  0.016321  0.104886
 2022-03-22 16:00:00  0.104387  0.000000  0.042461  0.000000
 ...                       ...       ...       ...       ...
 2023-03-21 00:00:00  0.578777  0.552395  0.596965  0.581573
 2023-03-21 04:00:00  0.581339  0.548460  0.598279  0.587442
 2023-03-21 08:00:00  0.587242  0.551626  0.604424  0.586022
 2023-03-21 12:00:00  0.586349  0.546051  0.600254  0.581255
 2023-03-21 16:00:00  0.580494  0.542702  0.589925  0.571050
 
 [2189 rows x 4 columns],
                          Open      High       Low     Close
 Time                                                       
 2022-03-22 00:00:00  0.274971  0.308707  0.237208  0.289

In [ ]:
feeds = [bt.feeds.PandasData(dataname=df) for df in dfs_fit]
# feed1 = bt.feeds.PandasData(dataname=df1, name='df1')


In [ ]:
class Bband(bt.Strategy):

    def __init__(self):
        self.inds = dict()
        self.pos = 0
        
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()
            bb = bt.ind.BollingerBands(d.close, period=20, devfactor=2.0)
            self.inds[d]['bb_upper'] = bb.lines.top
            self.inds[d]['bb_middle'] = bb.lines.mid
            self.inds[d]['bb_botton'] = bb.lines.bot

            self.inds[d]['rsi'] = bt.ind.RSI(d.close)
           
            
            # if i > 0: #Check we are not on the first loop of data feed:
            #     if self.p.oneplot == True:
            #         d.plotinfo.plotmaster = self.datas[0]
                    
                    
            # bb = bt.ind.BollingerBands(period=20, devfactor=2.0)
            # self.bb_upper = bb.lines.top
            # self.bb_middle = bb.lines.mid
            # self.bb_botton = bb.lines.bot

            

    def next(self):
        #  for i, d in enumerate(self.datas):
        # dt, dn = self.datetime.date(), d._name
        # pos = self.getpos(d).size
        # print('{} {} pos {}'.format(dt, dn, pos))
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
    
            values = (d.high, d.low,
                    d.close, d.open)
            
            max_value = max(*values)
            min_value = min(*values)

            if not self.pos and min_value < self.inds[d]['bb_botton'] and self.inds[d]['rsi'] < 30:
                self.pos = 1
                self.buy(data=d)  # enter long

            elif self.pos == 1 and max_value > self.inds[d]['bb_middle']:
                self.pos = 0
                self.close(data=d)  # close long pos

            if not self.pos and max_value > self.inds[d]['bb_upper'] and self.inds[d]['rsi'] > 70:
                self.pos = -1
                self.sell(data=d)  # enter short

            elif self.pos == -1 and min_value < self.inds[d]['bb_middle']:
                self.pos = 0
                self.close(data=d)  # close short pos


In [ ]:
cerebro = bt.Cerebro()
[cerebro.adddata(feed) for feed in feeds]
cerebro.addstrategy(Bband)

0

In [ ]:
cerebro.broker.setcommission(commission=0.0008)
# cerebro.broker.setcommission(margin=True, commission=0.0008, leverage=20)
cerebro.broker.set_cash(cash=1000)
# cerebro.addsizer(bt.sizers.PercentSizer, percents=100)
cerebro.addsizer(bt.sizers.FixedSize,stake=1000)

In [ ]:
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='areturn')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')


In [ ]:
output = cerebro.run()

In [ ]:

cerebro.plot(iplot=False, volume=False, )

[[<Figure size 640x480 with 20 Axes>]]

In [ ]:
output[0].analyzers.areturn.get_analysis()

OrderedDict([(2022, 7.646993788353164), (2023, 0.3278524968921599)])

In [ ]:
output[0].analyzers.returns.get_analysis()

OrderedDict([('rtot', 2.4407746948874163),
             ('ravg', 0.006687053958595661),
             ('rnorm', 4.39319297152337),
             ('rnorm100', 439.319297152337)])

In [ ]:
output[0].analyzers.drawdown.get_analysis()



AutoOrderedDict([('len', 145),
                 ('drawdown', 3.8764845021464036),
                 ('moneydown', 463.04520133234655),
                 ('max',
                  AutoOrderedDict([('len', 1100),
                                   ('drawdown', 108.75965565197002),
                                   ('moneydown', 6184.713363687785)]))])

In [ ]:
output[0].analyzers.sharpe.get_analysis()


OrderedDict([('sharperatio', 1.0868551334739192)])